<a href="https://colab.research.google.com/github/Beytullah-1001/Computer-Vision-For-Self-Driving-Cars/blob/main/Classifying%20Road%20Symbols/Classifying_Road_Symbols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import pickle
import pandas as pd
import random 
np.random.seed(0)

In [ ]:
!git clone https://bitbucket.org/jadslim/german-traffic-signs

In [ ]:
!ls german-traffic-signs

In [ ]:
!pip install keras


In [ ]:
with open('german-traffic-signs/train.p','rb') as f:
  train_data=pickle.load(f)
with open('german-traffic-signs/valid.p','rb') as f:
  valid_data=pickle.load(f)
with open('german-traffic-signs/test.p','rb') as f:
  test_data=pickle.load(f)

In [ ]:
print(type(train_data))
X_train,y_train=train_data["features"],train_data["labels"]
X_val,y_val=valid_data["features"],valid_data["labels"]
X_test,y_test=test_data["features"],test_data["labels"]

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
assert(X_train.shape[0]==y_train.shape[0]), "The number of images is not equal to the number of labels"
assert(X_val.shape[0]==y_val.shape[0]),"The number of images is not equal to the number of labels"
assert(X_test.shape[0]==y_test.shape[0]),"The number of images is not equal to the number of labels"
assert(X_train.shape[1:]==(32,32,3)),"The dimensions of the images are not 32x32x3"
assert(X_val.shape[1:]==(32,32,3)),"The dimensions of the images are not 32x32x3"
assert(X_test.shape[1:]==(32,32,3)),"The dimensions of the images are not 32x32x3"


In [ ]:
data=pd.read_csv('german-traffic-signs/signnames.csv')
data

In [ ]:
num_of_samples=[]


cols=5
num_classes=43

fig,axs=plt.subplots(nrows=num_classes,ncols=cols,figsize=(5,50))
fig.tight_layout()

for i in range(cols):
  for j,row in data.iterrows():
    x_selected=X_train[y_train==j]
    axs[j][i].imshow(x_selected[random.randint(0,len(x_selected)-1),:,:],cmap=plt.get_cmap('gray'))
    axs[j][i].axis("off")
    if i==2:
      axs[j][i].set_title(str(j)+"-"+row["SignName"])
      num_of_samples.append(len(x_selected))

In [ ]:
print(num_of_samples)
plt.figure(figsize=(12,4))
plt.bar(range(0,num_classes),num_of_samples)
plt.title("Distrubution of the train set")
plt.xlabel("Class number")
plt.ylabel("Number of images")
plt.show()



In [ ]:
plt.imshow(X_train[1000])
print(X_train[1000].shape)


In [ ]:
import cv2

In [ ]:
def cvrt_grayscale(img):
  img=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  return img

img=cvrt_grayscale(X_train[1000])
print(img.shape)
plt.imshow(img);
plt.axis("off")


In [ ]:
def equalize(img):
  img=cv2.equalizeHist(img)
  return img
img=equalize(img)
plt.imshow(img)

In [ ]:
def preprocessing(img):
  grayscale_image=cvrt_grayscale(img)
  equalized_image=equalize(grayscale_image)
  normalized_image=equalized_image/255
  return equalized_image

In [ ]:
X_train=np.array(list(map(preprocessing,X_train)))
X_val=np.array(list(map(preprocessing,X_val)))
X_test=np.array(list(map(preprocessing,X_test)))

In [ ]:
plt.imshow(X_train[random.randint(0,len(X_train)-1)])
print(X_train.shape)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
y_train=to_categorical(y_train,43)
y_val=to_categorical(y_val,43)
y_test=to_categorical(y_test,43)

## Lenet Implementation

In [ ]:
def modified_model():
  model = Sequential()
  model.add(Conv2D(60, (5, 5), input_shape=(32, 32, 1), activation='relu'))
  model.add(Conv2D(60, (5, 5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(30, (3, 3), activation='relu'))
  model.add(Conv2D(30, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(43, activation='softmax'))

  model.compile(Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model = modified_model()
print(model.summary())


In [ ]:
history=model.fit(X_train,y_train,epochs=10,batch_size=400,verbose=1,shuffle=1,validation_data=(X_val,y_val))

Epoch 1/10
87/87 [==============================] - 230s 3s/step - loss: 3.1171 - accuracy: 0.3311 - val_loss: 0.8581 - val_accuracy: 0.7707
Epoch 2/10
87/87 [==============================] - 228s 3s/step - loss: 0.6500 - accuracy: 0.8073 - val_loss: 0.3244 - val_accuracy: 0.9088
Epoch 3/10
65/87 [=====================>........] - ETA: 55s - loss: 0.3474 - accuracy: 0.8954

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(['training','validation'])
plt.xlabel(" Epoch")
plt.ylabel("Loss")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history["val_accuracy"])
plt.legend(['training','validation'])
plt.xlabel(" Epoch")
plt.ylabel("Loss")